In [1]:
import numpy as np
import matplotlib.pyplot as plt
from multiprocessing import Pool, cpu_count

from qTools.classes import *
from qTools.QuantumToolbox import *

# Define system

In [2]:
system = QuantumSystem(name = 'TestSystem')
qub = Qubit(frequency=0)
cav = Cavity(dimension=100, frequency=2)
coupling = system.JC(qub, cav, 1.79)
system.add(qub, cav)

Qubit requires a dimension
Cavity requires a dimension
sigmaz


# Define protocol

In [3]:
tau = 10e-3
protocol = Protocol(superSys=system)
JC = FreeEvolution(time=tau/2)
JC.createUpdate()
X = Gate(qub, sigmax)
protocol.add(JC, X, JC, JC, X, JC)
system.Unitaries = protocol.unitary

# Define Simulation/Sweep

In [4]:
sim = Simulation(qSys=system)
system.initialState = [0,0]
cavFreqSweep = sim.Loop.addSweep(sys=cav, sweepKey='frequency')
cavFreqSweep.sweepList = np.arange(-4, 4+0.05, 0.05)/2
sim.steps = 80

[0, 0]
creating state
here2


# Define compute functions

In [5]:
cavParity = parityEXP(cav.freeMat)
def expect(qSys, state):
    expect = expectationKet(cavParity, state)
    return expect
sim.compute = expect

# Run Simulation

In [6]:
p = Pool(processes=cpu_count())
statesDigit, parityDigit = sim.run()
p.close()
p.join()

TypeError: getattr(): attribute name must be string

In [ ]:
surf = plt.pcolormesh(np.transpose(parityDigit))
plt.colorbar(surf)
plt.show()

In [ ]:
protocol.steps

In [7]:
print(JC.updates)